# Baremetal Artifact Environment

This is a simple notebook that provisions a standard Ubuntu node on a bare metal host with an artifact repository cloned into the home directory of the `cc` user, ready for you to experiment.

The first thing you must do is set what site and project to use. You can select this from the dropdown that displays, or use the defaults that are automatically picked. For more information about setting context in python-chi, [see here](https://python-chi.readthedocs.io/en/dev/modules/context.html).

In [ ]:
from chi import context

# During the transition period, we need to opt into the some of the
# new python-chi functions. Otherwise the functional interface will
# return the old types.
context.version = "1.0"

context.choose_site(default="CHI@TACC")
context.choose_project()

### Check available hardware

Next, we'll pick which hardware to use. The following code cell looks for nodes of type matching the `node_type` variable, and filters our ones that are reserved.

This information comes from [the Chameleon hardware repository](https://chameleoncloud.org/hardware), and can be [queryed via python-chi](https://python-chi.readthedocs.io/en/dev/modules/hardware.html).

In [ ]:
from chi import hardware

node_type = "compute_cascadelake_r"
available_nodes = hardware.get_nodes(node_type=node_type, filter_reserved=True)
if available_nodes:
    print(f"There currently are {len(available_nodes)} {node_type} nodes ready to use")
    hardware.show_nodes(available_nodes)
else:
    print(f"All {node_type} nodes are in use! You could use next_free_timeslot to see how long you need to wait, or use the calendar.")

### Reserve node

In order to use hardware on Chameleon, you'll need to [make a reservation](https://chameleoncloud.readthedocs.io/en/latest/technical/reservations.html). You can do this via [the python-chi lease module](https://python-chi.readthedocs.io/en/dev/modules/lease.html).

If the resources you want to use are currently free, you can make a lease that starts right now. The following code does this to reserve one of the nodes found above, and it also reserves a floating ip.

In [ ]:
from chi import lease
from datetime import timedelta
import os

my_lease = lease.Lease(f"{os.getenv('USER')}-power-management", duration=timedelta(hours=3))
my_lease.add_node_reservation(nodes=[available_nodes[0]]) # or you could use node_type=node_type
my_lease.add_fip_reservation(1) # include a floating ip
my_lease.submit(idempotent=True)

### Create a server on the node

Next, we will launch the reserved node with [an image](https://chameleoncloud.readthedocs.io/en/latest/technical/images.html). You can search for [Chameleon supported images in python-chi](https://python-chi.readthedocs.io/en/dev/modules/image.html). Here, we use an Ubuntu image.

Then, we submit the request to create a server, using the reservation information from our lease. You'll need to wait for the server to fully provision, which can around 10 minutes depending on the node.

In [ ]:
from chi import server

my_server = server.Server(
    f"{os.getenv('USER')}-power-management",
    reservation_id=my_lease.node_reservations[0]["id"],
    image_name="CC-Ubuntu24.04",
)
my_server.submit(idempotent=True)

### Configure networking on the node

Now, we must configure the server to use the floating IP we reserved earlier. We'll also need to wait for the networking to finish configuring, which may take a few additional minutes.

In [ ]:
fip = my_lease.get_reserved_floating_ips()[0]
my_server.associate_floating_ip(fip)
my_server.check_connectivity(host=fip)

### Install software on the node

Now we will install our software on the node, over SSH. In your own experiment, you would likely want to change these commands.

You can use the `execute` [method](https://python-chi.readthedocs.io/en/dev/modules/server.html#chi.server.Server.execute) or for more advance usage you can [get a Fabric Connection object](https://python-chi.readthedocs.io/en/dev/modules/server.html#chi.server.Server.ssh_connection).

In [ ]:
# Clone git repo with experiment source code
my_server.execute("git clone https://github.com/ChameleonCloud/trovi_external_artifacts_deployment.git") # placeholder for now

# Run setup script
my_server.execute("bash trovi_external_artifacts_deployment/scripts/setup.sh")

## Experiment with the Artifact

Now, we can finally experiment with the artifact by SSHing directly to the node and running commands. You can also add to this notebook and run commands on the server similar to the previous step.